In [4]:
%%time

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
from scipy.stats import norm

# binomial_tree_option_priceModel
from scipy.stats import norm
import numpy as np

def binomial_tree_option_price(S, K, T, r, sigma, N=20_000, dividends=0, skew=0, kurtosis=0):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt) + skew * sigma * dt)  # Adjusted for skew
    d = np.exp(-sigma * np.sqrt(dt) + skew * sigma * dt)  # Adjusted for skew
    p = (np.exp((r - dividends) * dt) - d) / (u - d)  # Adjusted for dividends
    
    # Adjust p slightly to account for kurtosis (heuristic adjustment)
    p += kurtosis * 0.001 * (1 - 2 * p)  # This is a heuristic adjustment

    prices = S * d**np.arange(N, -1, -1) * u**np.arange(0, N+1, 1)
    option_values = np.maximum(prices - K, 0)
    
    discount_factor = np.exp(-r * dt)
    for i in range(N - 1, -1, -1):
        option_values[:i+1] = (p * option_values[1:i+2] + (1 - p) * option_values[:i+1]) * discount_factor
        option_values[:i+1] = np.maximum(option_values[:i+1], prices[:i+1] - K)
    
    return option_values[0]


# Retrieve historical data
ticker = "BTCC-B.TO"
start_time = (datetime.now(pytz.timezone('US/Pacific')) - timedelta(days=365*4)).strftime('%Y-%m-%d')
end_time = (datetime.now(pytz.timezone('US/Pacific'))).strftime('%Y-%m-%d')
data = yf.download(ticker, start=start_time, end=end_time, interval="1d")[['Close']]
data['Daily_Return'] = data['Close'].pct_change()
daily_volatility = data['Daily_Return'].std()
annualized_volatility = daily_volatility * np.sqrt(252)

# Risk-free rate (annualized)
risk_free_rate = 0.05  # Example rate, adjust as needed

# Current stock price
current_stock_price = data['Close'].iloc[-1]

# Calculate the next 4 Fridays from today, spaced two weeks apart
today = datetime.now(pytz.timezone('US/Pacific')).date()
friday_dates = [today + timedelta(days=((4 - today.weekday()) % 7) + 30*i) for i in range(12)]

# Generate a range of strike prices from -50% to +50% of the current stock price by 5% increments
strike_price_range = np.arange(0, 0.8, 0.05)  # Ends at 0.55 to include 50%
strike_prices = [current_stock_price * (1 + x) for x in strike_price_range]

# Initialize DataFrame to store results
columns = ['Expiration Date', 'Strike Price', 'Call Option Price']
results_df = pd.DataFrame(columns=columns)

# Calculate option prices, storing results in the DataFrame
for expiry_date in friday_dates:
    days_to_expiration = (expiry_date - today).days / 365  # Convert days to years
    for strike_price in strike_prices:
        option_price = binomial_tree_option_price(current_stock_price, strike_price, days_to_expiration, risk_free_rate, annualized_volatility)
        
        # Append results to the DataFrame
        result = pd.DataFrame([[expiry_date, strike_price, option_price]], columns=columns)
        results_df = pd.concat([results_df, result], ignore_index=True)


# Save the results DataFrame to a CSV file
results_df.to_csv('option_prices.csv', index=False)

# Optionally, you can print a message to confirm that the data was saved successfully
print("Option prices saved to 'option_prices.csv'")


[*********************100%%**********************]  1 of 1 completed
<timed exec>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:16: RuntimeWarning: invalid value encountered in scalar divide
<timed exec>:67: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


Option prices saved to 'option_prices.csv'
CPU times: total: 1min 22s
Wall time: 1min 22s
